In [3]:
import cv2
import pandas as pd
import numpy as np
import time
from pyzbar.pyzbar import decode
from datetime import datetime
import math

def simulate_realtime_update(interval_sec=10, duration_sec=60):
    global df
    start_time = time.time()
    while (time.time() - start_time) < duration_sec:
        today = pd.Timestamp('2025-05-31')
        df = update_status(df, today)
        inventory_table, fresh_table, discount_table, donation_table, expired_table = generate_tables(df)

        print(f"\n🕒 Updated @ {datetime.now().strftime('%H:%M:%S')} (System Date: {today.date()})")
        print(f"Fresh: {len(fresh_table)} | Discount: {len(discount_table)} | Donation: {len(donation_table)} | Expired: {len(expired_table)}")
        time.sleep(interval_sec)

# --- Load Initial Data ---
df = pd.read_csv("grocery_df.csv")

# Convert necessary date columns
for col in ['Date_Received', 'Expiration_Date', 'Discount_Date', 'Donation_Date']:
    df[col] = pd.to_datetime(df[col])

# --- Status Function ---
def get_status(row, today):
    if today < row['Discount_Date']:
        return 'Fresh'
    elif row['Discount_Date'] <= today < row['Donation_Date']:
        return 'Discount'
    elif row['Donation_Date'] <= today < row['Expiration_Date']:
        return 'Donation'
    else:
        return 'Expired'

def update_status(df, today):
    df['Status'] = df.apply(lambda row: get_status(row, today), axis=1)
    return df

# --- Table Generator ---
def generate_tables(df):
    inventory_table = df[['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name',
                          'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity', 'Unit_Price',
                          'Date_Received', 'Expiration_Date', 'Discount_Date', 'Donation_Date',
                          'Shelf_Life_(Days)', 'Shelf_Life_(Years)', 'Shelf_Life_Bin', 'Status']]

    fresh_table = inventory_table[df['Status'] == 'Fresh']
    discount_table = inventory_table[df['Status'] == 'Discount']
    donation_table = inventory_table[df['Status'] == 'Donation']
    expired_table = inventory_table[df['Status'] == 'Expired']

    return inventory_table, fresh_table, discount_table, donation_table, expired_table

# --- Refresh all tables ---
def refresh_all_tables(current_date):
    global df, inventory_table, fresh_table, discount_table, donation_table, expired_table
    df = update_status(df, pd.Timestamp(current_date))
    inventory_table, fresh_table, discount_table, donation_table, expired_table = generate_tables(df)

# --- Add new product ---
def add_new_item_via_qr(data):
    global df
    # Convert dates
    data['Date_Received'] = pd.to_datetime(data['Date_Received'])
    data['Expiration_Date'] = pd.to_datetime(data['Expiration_Date'])

    # Swap dates if needed
    if data['Expiration_Date'] < data['Date_Received']:
        data['Date_Received'], data['Expiration_Date'] = data['Expiration_Date'], data['Date_Received']

    # Shelf life
    shelf_life_days = (data['Expiration_Date'] - data['Date_Received']).days
    shelf_life_years = round(shelf_life_days / 365, 2)

    # Bin
    bin_label = '0-1 years' if shelf_life_years < 1 else f'{int(shelf_life_years)}-{int(shelf_life_years)+1} years'

    # Discount/Donation
    discount_fraction = 0.2 if bin_label == '0-1 years' else 0.1
    donation_fraction = 0.1 if bin_label == '0-1 years' else 0.05

    discount_date = data['Expiration_Date'] - pd.to_timedelta(shelf_life_days * discount_fraction, unit='D')
    donation_date = data['Expiration_Date'] - pd.to_timedelta(shelf_life_days * donation_fraction, unit='D')

    # Add columns
    data['Shelf_Life_(Days)'] = shelf_life_days
    data['Shelf_Life_(Years)'] = shelf_life_years
    data['Shelf_Life_Bin'] = bin_label
    data['Discount_Date'] = discount_date
    data['Donation_Date'] = donation_date

    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
    print(f"✅ Added new item via QR: {data['Product_Name']}")

# --- QR Scanner ---
cap = cv2.VideoCapture(0)
print("Press 's' to scan QR code. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("QR Scanner", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        print("Exiting scanner...")
        break

    elif key == ord('s'):
        decoded = decode(frame)
        if not decoded:
            print("❌ No QR code found. Try again.")
            continue

        for barcode in decoded:
            qr_text = barcode.data.decode('utf-8')
            print("Scanned QR:", qr_text)

            try:
                # Parse QR string into dictionary
                scanned_info = dict(pair.split(':') for pair in qr_text.split('|'))

                # Clean up types
                scanned_info['Unit_Price'] = float(scanned_info.get('Unit_Price', '0').replace('$', '').strip())
                scanned_info['Stock_Quantity'] = int(scanned_info.get('Stock_Quantity', 0))
                scanned_info['Reorder_Level'] = int(scanned_info.get('Reorder_Level', 0))
                scanned_info['Reorder_Quantity'] = int(scanned_info.get('Reorder_Quantity', 0))

                # Convert date fields
                scanned_info['Date_Received'] = pd.to_datetime(scanned_info['Date_Received'])
                scanned_info['Expiration_Date'] = pd.to_datetime(scanned_info['Expiration_Date'])

                # Add new item
                if scanned_info not in df.to_dict('records'):
                    add_new_item_via_qr(scanned_info)
                    refresh_all_tables(current_date='2025-05-31')
                    print("✅ Product added and tables updated.")
                else:
                    print("⚠️ This QR code has already been added.")

            except Exception as e:
                print("❌ Error parsing QR:", e)

cap.release()
cv2.destroyAllWindows()

# Save after scanning
df.to_csv('inventory.csv', index=False)
print("✅ Table saved to inventory.csv")

# --- Start Real-Time Simulation ---
simulate_realtime_update(interval_sec=5, duration_sec=20)


Press 's' to scan QR code. Press 'q' to quit.
❌ No QR code found. Try again.
❌ No QR code found. Try again.
❌ No QR code found. Try again.
Scanned QR: Product_ID:40-681-9341|Product_Name:Random thing|Catagory:Dairy|Supplier_ID:69-933-2582|Supplier_Name:Realpoint|Stock_Quantity:123|Reorder_Level:90|Reorder_Quantity:99|Unit_Price:$0.80|Date_Received:2025-01-26|Expiration_Date:2025-06-05
✅ Added new item via QR: Random thing
✅ Product added and tables updated.
Exiting scanner...
✅ Table saved to inventory.csv

🕒 Updated @ 14:31:07 (System Date: 2025-05-31)
Fresh: 323 | Discount: 56 | Donation: 46 | Expired: 565

🕒 Updated @ 14:31:12 (System Date: 2025-05-31)
Fresh: 323 | Discount: 56 | Donation: 46 | Expired: 565

🕒 Updated @ 14:31:17 (System Date: 2025-05-31)
Fresh: 323 | Discount: 56 | Donation: 46 | Expired: 565

🕒 Updated @ 14:31:22 (System Date: 2025-05-31)
Fresh: 323 | Discount: 56 | Donation: 46 | Expired: 565


,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin,Discount_Date,Donation_Date,Status
985,62-393-9939,Cheddar Cheese,Dairy,93-877-9384,Gabcube,60,9,89,$9.00,2025-01-01,2025-05-05,126,0.35,0-1 years,2025-04-09 19:12:00,2025-04-22 09:36:00,Expired
986,31-745-6850,Cabbage,Fruits & Vegetables,96-215-2767,Lajo,94,90,12,$0.90,2025-05-03,2025-06-01,29,0.08,0-1 years,2025-05-26 04:48:00,2025-05-29 02:24:00,Donation
987,86-692-2312,Avocado Oil,Oils & Fats,77-783-4107,Dazzlesphere,30,48,52,$10.00,2024-11-30,2025-01-11,42,0.12,0-1 years,2025-01-02 14:24:00,2025-01-06 19:12:00,Expired
988,28-044-4102,Papaya,Fruits & Vegetables,93-358-1118,Yakijo,19,28,83,$4.50,2025-03-27,2025-06-26,91,0.25,0-1 years,2025-06-07 19:12:00,2025-06-16 21:36:00,Fresh
989,40-681-9341,Random thing,Dairy,69-933-2582,Realpoint,123,90,99,0.8,2025-01-26,2025-06-05,130,0.36,0-1 years,2025-05-10 00:00:00,2025-05-23 00:00:00,Donation


,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin,Discount_Date,Donation_Date,Status
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2025-03-16,2025-04-19,34,0.09,0-1 years,2025-04-12 04:48:00,2025-04-15 14:24:00,Expired
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-12-08,2025-06-01,177,0.48,0-1 years,2025-04-26 14:24:00,2025-05-14 07:12:00,Donation
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2025-03-03,2025-04-22,50,0.14,0-1 years,2025-04-12 00:00:00,2025-04-17 00:00:00,Expired
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-11-17,2025-07-08,235,0.64,0-1 years,2025-05-22 00:00:00,2025-06-14 12:00:00,Discount
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2025-02-03,2025-05-05,94,0.26,0-1 years,2025-04-16 04:48:00,2025-04-25 14:24:00,Expired


,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Discount_Date,Donation_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin,Status
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2025-03-16,2025-04-19,2025-04-12 04:48:00,2025-04-15 14:24:00,34,0.09,0-1 years,Expired
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2025-03-03,2025-04-22,2025-04-12 00:00:00,2025-04-17 00:00:00,50,0.14,0-1 years,Expired
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2025-02-03,2025-05-05,2025-04-16 04:48:00,2025-04-25 14:24:00,94,0.26,0-1 years,Expired
6,71-516-1996,Corn Oil,Oils & Fats,04-391-7610,Tagfeed,96,52,16,$2.50,2024-10-18,2025-01-20,2025-01-01 04:48:00,2025-01-10 14:24:00,94,0.26,0-1 years,Expired
7,39-629-5554,Egg (Goose),Dairy,67-679-4930,Muxo,44,90,17,$2.50,2025-02-03,2025-02-05,2025-02-04 14:24:00,2025-02-04 19:12:00,2,0.01,0-1 years,Expired


,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Discount_Date,Donation_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin,Status
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-12-08,2025-06-01,2025-04-26 14:24:00,2025-05-14 07:12:00,177,0.48,0-1 years,Donation
16,55-936-2406,Bread Flour,Grains & Pulses,29-689-5014,Mynte,27,93,88,$1.50,2024-11-05,2025-06-16,2025-05-02 00:00:00,2025-05-24 12:00:00,225,0.62,0-1 years,Donation
26,02-508-3777,Egg (Quail),Dairy,69-933-2582,Realpoint,97,88,81,$0.80,2025-01-26,2025-06-05,2025-05-09 14:24:00,2025-05-22 19:12:00,132,0.36,0-1 years,Donation
33,79-136-9840,Herbal Tea,Beverages,07-809-0926,Pixonyx,77,45,75,$30.00,2025-01-23,2025-06-05,2025-05-09 00:00:00,2025-05-22 12:00:00,135,0.37,0-1 years,Donation
45,90-230-9767,Long Grain Rice,Grains & Pulses,70-515-9048,Voonyx,67,88,32,$1.50,2024-11-06,2025-06-17,2025-05-03 00:00:00,2025-05-25 12:00:00,225,0.62,0-1 years,Donation


,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Discount_Date,Donation_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin,Status
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-11-17,2025-07-08,2025-05-22 00:00:00,2025-06-14 12:00:00,235,0.64,0-1 years,Discount
9,46-452-9419,Egg (Duck),Dairy,67-137-4215,Wordify,43,10,15,$1.00,2025-02-08,2025-06-18,2025-05-22 09:36:00,2025-06-04 16:48:00,133,0.36,0-1 years,Discount
40,11-316-8405,Mango,Fruits & Vegetables,50-918-1373,Twitternation,24,55,51,$5.00,2024-11-26,2025-06-25,2025-05-13 09:36:00,2025-06-03 16:48:00,213,0.58,0-1 years,Discount
54,36-918-2937,Green Coffee,Beverages,53-844-0478,Kwinu,62,89,18,$12.00,2024-10-12,2025-06-28,2025-05-06 19:12:00,2025-06-01 21:36:00,261,0.72,0-1 years,Discount
66,20-225-3930,White Tea,Beverages,73-069-2753,Skipstorm,99,87,10,$25.00,2024-10-17,2025-07-08,2025-05-15 19:12:00,2025-06-11 09:36:00,266,0.73,0-1 years,Discount
